In [1]:
import pandas as pd
import numpy as np
import random
import math

In [2]:
schedule = pd.read_csv('./data/clean_timetable.csv')
rooms = pd.read_csv('./data/clean_rooms.csv')

In [3]:
print("There's %i type of classes" %(len(schedule['Unidade de execução'].unique())))
print("There's %i available rooms" % (rooms.shape[0]))

There's 759 type of classes
There's 120 available rooms


In [66]:
# schedule[schedule['Sala da aula'].isnull()]

In [4]:
# schedule

In [5]:
# schedule.groupby(['Unidade de execução']).size()

In [5]:
df_count_class = schedule[['Unidade de execução', 'Turma', 'Turno']].groupby(['Unidade de execução','Turma', 'Turno'], dropna=False)['Unidade de execução'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False)
                             #.reset_index(drop=True)

In [6]:
# for row in df_count_class[df_count_class['Turma'].isnull()].values:
#     if math.isnan(row[1]):
#         print('nan')

In [43]:
df_mean_class = schedule[['Unidade de execução', 'Inscritos no turno (no 1º semestre é baseado em estimativas)', 'Turma']].groupby(['Unidade de execução', 'Turma'], dropna=False)['Inscritos no turno (no 1º semestre é baseado em estimativas)'] \
                             .mean().round(0) \
                             .reset_index(name='mean')

In [46]:
# df_mean_class[df_mean_class['Unidade de execução'] == "Redes Digitais II - Sistemas, Aplicações e Serviços"]

In [47]:
# schedule[(schedule['Unidade de execução'] == "Redes Digitais II - Sistemas, Aplicações e Serviços") & (schedule['Turma'] == "ET-PLC1")]

In [45]:
# schedule[schedule['Code'] == 13377]

In [8]:
# df_mean_class[df_mean_class['Unidade de execução'] == 'Contabilidade Financeira I'] #Test purposes

In [8]:
def get_rooms(class_capacity_level):
    capacity_to_filter = class_capacity_level - 5
    rooms_filtered = rooms[(rooms['Capacidade Normal'] >= capacity_to_filter)].sort_values(by=['Capacidade Normal'], ascending=True)
    return rooms_filtered['Code'].reset_index(drop=True)

In [48]:
# def choose_best_room(rooms_timetable, array_rooms, class_init):
#     available_rooms = []
#     for index in range(len(array_rooms)):
#         room = array_rooms[index]
#         timetable = rooms_timetable[room]
#         intersection_time = np.intersect1d(timetable, class_init, assume_unique=False, return_indices=False)
#         if len(intersection_time) != 0:
#             available_rooms.append(index)
#     if len(available_rooms) == 0:
#         print('EMPTY ROOMS')
#     filtered_rooms = np.take(array_rooms, available_rooms).reset_index(drop=True) if len(available_rooms) != 0 else array_rooms
#     choose_random_room = False if len(available_rooms) == 0 else True #in development
#     random_index = random.randint(0, len(filtered_rooms)-1)
#     return filtered_rooms[random_index] if choose_random_room == True else filtered_rooms[0]

In [10]:
# def choose_best_room(rooms_timetable, array_rooms, class_init):
#     available_rooms = []
#     n_classes = len(class_init)
#     times_assign = class_init
#     for index in range(len(array_rooms)):
#         room = array_rooms[index]
#         timetable = rooms_timetable[room]
#         intersection_time = np.intersect1d(timetable, times_assign, assume_unique=False, return_indices=False)
#         if len(intersection_time) != 0:
#             available_rooms.append(index)
#         else: # Gotta assign other room for this time
#             times_assign = intersection_time
#     if len(available_rooms) == 0:
#         print('EMPTY ROOMS')
#     filtered_rooms = np.take(array_rooms, available_rooms).reset_index(drop=True) if len(available_rooms) != 0 else array_rooms
#     choose_random_room = False if len(available_rooms) == 0 else True #in development
#     random_index = random.randint(0, len(filtered_rooms)-1)
#     return filtered_rooms[random_index] if choose_random_room == True else filtered_rooms[0]

In [49]:
def choose_best_room_oficial(rooms_timetable, array_rooms, class_init, name_class, c_class, turn_class):
    available_rooms = []
    solution = pd.DataFrame(columns=["Time", "Room Code"])
    n_classes = len(class_init)
    times_assign = class_init['Início'].values.tolist()
    aux_counter = 1
    for index in range(len(array_rooms)):
        room = array_rooms[index]
        timetable = rooms_timetable[room]
        intersection_time = np.intersect1d(timetable, times_assign, assume_unique=False, return_indices=False).tolist()
        if len(intersection_time) == 0:
            data = []
            for time in times_assign:
                data.append((time, room))
            aux_df = pd.DataFrame(data=data, columns=["Time", "Room Code"])
            solution = solution.append(aux_df, ignore_index=True)
            n_classes = n_classes - len(times_assign)
            if n_classes == 0:
                break
        elif len(intersection_time) != len(timetable): # Didn't intersect any time --> so is empty
            #print("Didn't find time")
            available_times = np.setdiff1d(times_assign, intersection_time, assume_unique=False).tolist()
            data = []
            for time in available_times:
                data.append((time, room))
            aux_df = pd.DataFrame(data=data, columns=["Time", "Room Code"])
            solution = solution.append(aux_df, ignore_index=True)
            n_classes = len(intersection_time)
            times_assign = intersection_time
            if n_classes == 0:
                times_assign = []
                break
        aux_counter +=1
    #Verify if still has classes that has no assigned room
    final_solution = class_init.merge(solution, how="inner", left_on="Início", right_on="Time")[['Code', 'Início' ,"Room Code"]]
    return final_solution

In [50]:
def update_timetable(df_solution, timetable): #selected_room variable--> input
    timetable_c = timetable.copy()
    list_rooms = df_solution['Room Code'].unique()
    for room_code in list_rooms:
        timetable_c[room_code] = timetable_c[room_code] + df_solution[df_solution['Room Code'] == room_code]['Início'].values.tolist()
    return timetable_c

In [51]:
# IN DEVELOPMENT
rooms_timetable = [[]] * len(rooms)
df_solution = pd.DataFrame(columns=["Code", 'Início' ,"Room Code"])
counter = 0
for row in df_count_class.values:
    name_class = row[0]
    c_class = row[1]
    turn_class = row[2]
    class_timetable = schedule[(schedule['Unidade de execução'] == name_class) & (schedule['Turma'] == c_class) & (schedule['Turno'] == turn_class)][['Code', 'Início']] if isinstance(c_class, str) else schedule[(schedule['Unidade de execução'] == name_class) & (schedule['Turma'].isnull()) & (schedule['Turno'] == turn_class)][['Code', 'Início']]
    n_students = int(df_mean_class[(df_mean_class['Unidade de execução'] == name_class) & (df_mean_class['Turma'] == c_class)]['mean']) if isinstance(c_class, str) else int(df_mean_class[(df_mean_class['Unidade de execução'] == name_class) & (df_mean_class['Turma'].isnull())]['mean'])
    # Get list of rooms that can handle the n_students
    list_rooms = get_rooms(n_students)
    # Select the first room of the list
    selected_room = choose_best_room_oficial(rooms_timetable, list_rooms, class_timetable, name_class, c_class, turn_class)
    #print(selected_room)
    # Assign selected room for that class
    #solution[counter] = selected_room
    df_solution = df_solution.append(selected_room)
    #print("\n")
#     print(df_solution)
    # Update array_timetable
    #rooms_timetable[selected_room] = rooms_timetable[selected_room] + class_timetable
    rooms_timetable = update_timetable(selected_room, rooms_timetable)
    counter +=1
df_solution = df_solution.drop_duplicates()
# Final result should a dataframe that contains assigned room for each class
# Join or create a new array to make the join

In [53]:
# int(df_mean_class[(df_mean_class['Unidade de execução'] == "Tese em Serviço Social I") & (df_mean_class['Turma'].isnull())]['mean'])

In [52]:
# schedule[(schedule['Code'] == 15464)]

In [54]:
# df_solution

In [142]:
# intersect = pd.concat([schedule['Code'], df_solution['Code']]).drop_duplicates(keep=False)

In [55]:
# intersect

In [56]:
# schedule[schedule['Code'] == 23674] 

In [57]:
# df_count_class[df_count_class['Unidade de execução'] == "Jornalismo de Dados e Visualização de Informação"]

In [58]:
# len(df_count_class['Unidade de execução'].unique())

In [59]:
# len(schedule['Unidade de execução'].unique())

In [60]:
# final_solution = df_class_init.merge(df_solution, how="inner", left_on="Início", right_on="Time")#[['Code', "Room Code"]]
# final_solution[final_solution.duplicated(['Code'])]

In [21]:
print('Solution length: %i ' % (len(df_solution)))
print('Needed length: %i ' % (len(schedule)))
print('Number duplicated code: %i' % (len(df_solution) - len(df_solution['Code'].unique())))

Solution length: 23957 
Needed length: 23957 
Number duplicated code: 0


In [31]:
def merge_solution(df_schedule, df_solution, df_rooms):
    df_sol_smp = df_solution.merge(df_rooms, how="inner", left_on="Room Code", right_on="Code")[['Code_x', 'Nome sala', 'Capacidade Normal']].rename(columns={"Code_x": "Class Code"})
    interesting_columns = df_schedule.columns.tolist() + ["Nome sala", "Capacidade Normal", "Lotação_Default"]
    interesting_columns.remove("Code")
    interesting_columns.remove("Sala da aula")
    interesting_columns.remove("Lotação")
    df_join = df_schedule.merge(df_sol_smp, how="inner", left_on="Code", right_on="Class Code").rename(columns={"Lotação": "Lotação_Default"})
    df_final = df_join[interesting_columns].rename(columns={"Nome sala": "Sala da aula", "Capacidade Normal": "Lotação"}).to_csv("./output/final_schedule2.csv", index=False, encoding="utf-8-sig")

In [32]:
merge_solution(schedule, df_solution, rooms)

In [61]:
# schedule['Unidade de execução'].unique()

In [62]:
# schedule[(schedule['Unidade de execução'] == "Redes Digitais II - Sistemas, Aplicações e Serviços") & (schedule['Inscritos no turno (no 1º semestre é baseado em estimativas)'] == 16)]

In [179]:
# schedule[schedule['Code'] == 15119]

In [177]:
# df_solution[df_solution['Code'] == 15119]

In [178]:
# df_solution.groupby(['Code'])['Code'].count().sort_values(ascending=False)

In [44]:
# df_test = schedule[schedule['Unidade de execução'] == "Contabilidade Financeira I"]

In [53]:
# df_cf = schedule[schedule['Unidade de execução'] == "Contabilidade Financeira I"]

In [174]:
# df_cf[df_cf['Início'] == "01/12/2015 08:00:00"]

In [175]:
# df_test.groupby(['Início'])['Início'].count().sort_values(ascending=False).head(10)

In [176]:
# df_solution['Code']

In [63]:
# rooms_timetable = [[]] * len(rooms)
# solution = [None] * len(df_count_class)
# counter = 0
# for row in df_count_class.values:
#     name_class = row[0]
#     class_timetable = schedule[schedule['Unidade de execução'] == name_class]['Início'].values.tolist()
#     n_students = int(df_mean_class[df_mean_class['Unidade de execução'] == name_class]['mean'])
#     # Get list of rooms that can handle the n_students
#     list_rooms = get_rooms(n_students)
#     # Select the first room of the list
#     selected_room = choose_best_room(rooms_timetable, list_rooms, class_timetable)
#     # Assign selected room for that class
#     solution[counter] = selected_room
#     # Update array_timetable
#     rooms_timetable[selected_room] = rooms_timetable[selected_room] + class_timetable
#     counter +=1
# # Final result should a dataframe that contains assigned room for each class
# # Join or create a new array to make the join

In [38]:
# solution

In [39]:
# rooms_timetable

In [64]:
# df_solution.drop_duplicates()

In [65]:
# df_mean_class['Assigned Room'] = solution 

In [146]:
df_mean_class

,Unidade de execução,mean,Assigned Room
0,A Europa e o Mundo Entre as Guerras,36.0,4
1,A Europa e o Mundo no Século XIX,15.0,4
2,A Investigação Sociológica em Portugal,9.0,4
3,Abordagens à Psicopatologia,47.0,81
4,Acompanhamento de Obra e Fotogrametria,26.0,81
...,...,...,...
754,"Álgebra Linear, Geometria Analítica e Análise ...",56.0,76
755,Ética e Administração - Nível B,17.0,76
756,Ética e Deontologia Profissional,23.0,81
757,Ética e Desenvolvimento Profissional,13.0,18


In [120]:
array_test = []
for row in schedule.values:
    class_name = row[1]
    room_code = int(df_mean_class[df_mean_class['Unidade de execução'] == class_name]['Assigned Room'])
    array_test.append(room_code)

In [147]:
schedule['Assigned Room'] = array_test

In [12]:
def concat_solution_schedule(df_mean_class, df_schedule, solution):
    df = df_schedule.copy()
    df_mean_class['Assigned Room'] = solution
    array_solution = []
    for row in df.values:
        class_name = row[1]
        room_code = int(df_mean_class[df_mean_class['Unidade de execução'] == class_name]['Assigned Room'])
        array_solution.append(room_code)
    df['Assigned Room'] = array_solution
    return df

In [180]:
def generate_final_schedule(df_schedule, df_rooms):
    interesting_columns = df_schedule.columns.tolist() + ["Nome sala", "Capacidade Normal", "Lotação_Default"]
    interesting_columns.remove("Code")
    interesting_columns.remove("Assigned Room")
    interesting_columns.remove("Sala da aula")
    interesting_columns.remove("Lotação")
    df_join = df_schedule.merge(df_rooms, how="inner", left_on="Assigned Room", right_on="Code").rename(columns={"Lotação": "Lotação_Default"})
    df_final = df_join[interesting_columns].rename(columns={"Nome sala": "Sala da aula", "Capacidade Normal": "Lotação"}).to_csv("./output/final_schedule2.csv", index=False, encoding="utf-8-sig")

In [181]:
generate_final_schedule(schedule, rooms)

In [178]:
df_final.to_csv("./output/final_schedule2.csv", index=False, encoding="utf-8-sig")

In [143]:
def generate_final_solution(df_schedule, df_rooms):
    all_classes = pd.read_csv('./data/clean_timetable.csv')
    all_columns = all_classes.columns
    df_final_schedule = df_schedule.merge(df_rooms, how="inner",left_on='Assigned Room Code', right_on='Code').rename(columns={'Unidade de execução_x': 'Unidade de execução', 'Início_x': 'Início', 'Fim_x': 'Fim','Dia_x': 'Dia','Edifício': 'Edifício', 'Número Horas_x': 'Número Horas', 'Inscritos no turno (no 1º semestre é baseado em estimativas)_x': 'Inscritos no turno (no 1º semestre é baseado em estimativas)', 'Lotação_y': 'Lotação', 'Lotação_x': 'Lotação_Default', 'Sala da aula_y': 'Sala da aula'})
    oficial_columns = all_columns.tolist() + ['Lotação_Default']
    return df_final[oficial_columns].drop(['Número Horas', 'Semana', 'Ano', 'Code'], axis=1)#.to_csv("./output/final_schedule.csv", index=False, encoding="utf-8-sig")

In [144]:
generated_schedule = generate_final_solution(df_final, rooms)

KeyError: "['Dia da Semana', 'Características reais da sala', 'Turno com inscrições superiores à capacidade das salas', 'Características da sala pedida para a aula', 'Semana', 'Turnos com capacidade superior à capacidade das características das salas', 'Ano', 'Lotação', 'Code', 'Sala da aula', 'Curso', 'Turno', 'Turma', 'Lotação_Default'] not in index"

In [134]:
df_final2 = df_final.merge(rooms, how="inner",left_on='Code', right_on='Code').rename(columns={'Unidade de execução_x': 'Unidade de execução', 'Início_x': 'Início', 'Fim_x': 'Fim','Dia_x': 'Dia','Edifício': 'Edifício', 'Número Horas_x': 'Número Horas', 'Inscritos no turno (no 1º semestre é baseado em estimativas)_x': 'Inscritos no turno (no 1º semestre é baseado em estimativas)', 'Lotação_y': 'Lotação', 'Lotação_x': 'Lotação_Default', 'Sala da aula_y': 'Sala da aula'})

In [137]:
df_final2['Nome sala_y']

0        C6.02
1        C6.02
2        C6.02
3        C6.02
4        C6.02
         ...  
23952    D0.03
23953    D0.03
23954    D0.03
23955    D0.03
23956    D0.03
Name: Nome sala_y, Length: 23957, dtype: object